In [1]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
import glob
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
### Load all emails and labels in a data frame ###

data_train = pd.DataFrame(columns=['X', 'y'])
data_test = pd.DataFrame(columns=['X'])

count_train = 0
count_test = 0

#Iterate over the 9 folds to get the training set
for i in range(1, 10):
    #Load spam emails
    for spam_path in glob.glob('lingspam_public/lemm_stop/part'+str(i)+'/spm*.txt'):
        with open(spam_path, 'r') as spam_file:
            opened_spam_file = spam_file.read().replace('\n', '')
            data_train.loc[count_train] = [opened_spam_file, 1]
            count_train += 1
            
    #Load legitimate emails
    for legit_path in glob.glob('lingspam_public/lemm_stop/part'+str(i)+'/[!spm]*.txt'):
        with open(legit_path, 'r') as legit_file:
            opened_legit_file = legit_file.read().replace('\n', '')
            data_train.loc[count_train] = [opened_legit_file, 0]
            count_train += 1

#Get the evaluation set under the eval folder
for eval_path in sorted(glob.glob('eval/all/*.txt')):
    with open(eval_path, 'r') as eval_file:
        opened_eval_file = eval_file.read().replace('\n', '')
        data_test.loc[count_test] = [opened_eval_file]
        count_test += 1        

In [3]:
MultinomialTF_1000 = Pipeline([('vectorizer', CountVectorizer()),
                               ('feature_selector', SelectKBest(mutual_info_classif, k=1000)), 
                               ('classifier', MultinomialNB())])
MultinomialTF_1000.fit(data_train['X'], data_train['y'].tolist())
y_test_pred = MultinomialTF_1000.predict(data_test['X'])

In [4]:
file = open('eval/results.txt','w') 
for item in y_test_pred:
    file.write("%s\n" % item)
file.close()